In [ ]:
DATASET_NAME = "5.COVID19_INSPECT"

In [ ]:
# Import libraries

import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%matplotlib inline
import numpy as np
from skimage.transform import resize
import time
from tqdm import tqdm
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau
from keras import callbacks
from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from keras.models import load_model
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

import models as M
import losses as l

ROOT_DIR = os.path.abspath("./")
LOG_PATH = os.path.join(ROOT_DIR, "logs")
TARGET_PATH = os.path.join(LOG_PATH, "{}".format(DATASET_NAME))
print(os.listdir(TARGET_PATH))

In [ ]:
# Save images here
try:
    # directory to create
    os.mkdir(os.path.join(TARGET_PATH, "images"))
except FileExistsError:
    # directory already exists
    pass

imgs_path = os.path.join(TARGET_PATH, "images")
imgs_path

In [ ]:
def read_txt(path, loss):
    
    data = []
    with open(path, 'r') as f:
        for line in f:
            
            if loss != True:
                line = float(line[:4]) * 10
            else:
                line = float(line[:4]) / 10
            
            data.append(line)
        f.close()
    
    return data

### Cross validation best fold plotted

In [ ]:
# Read Jaccard and Loss
ujp = read_txt(os.path.join(TARGET_PATH, "COVID19_Unet2_jacard.txt"), loss=False)
ulp = read_txt(os.path.join(TARGET_PATH, "COVID19_Unet2_loss.txt"), loss=True)
gjp = read_txt(os.path.join(TARGET_PATH, "COVID19_HPF_Unet3_jacard.txt"), loss=False)
glp = read_txt(os.path.join(TARGET_PATH, "COVID19_HPF_Unet3_loss.txt"), loss=True)

len(ujp), len(ulp), len(gjp), len(glp)

In [ ]:
interval = min(len(ujp), len(gjp))
epochs = range(interval)
plt.plot(epochs, ujp[:interval], 'r')
plt.plot(epochs, gjp[:interval], 'b')

#plt.title('CT Multiclass Segmentation Dataset: COVID-19 Radiology')
ft = 18
plt.ylabel('Jaccard Index (%)', fontsize=ft)
plt.xlabel('Epochs', fontsize=ft)
plt.yticks(fontsize=ft)
plt.xticks(fontsize=ft)
plt.legend(['U-Net', 'Sharp U-Net'], loc='lower right', fontsize=ft)
plt.savefig('{}/COVID_jgraph.pdf'.format(TARGET_PATH), bbox_inches='tight', pad_inches=0, dpi=300)

In [ ]:
interval = min(len(ulp), len(glp))
epochs = range(interval)
plt.plot(epochs, ulp[:interval], 'g')
plt.plot(epochs, glp[:interval], 'y')

plt.title('CT Multiclass Segmentation Dataset: COVID-19 Radiology')
plt.ylabel('Cross Entropy Loss')
plt.xlabel('Epochs')
plt.legend(['U-Net', 'Edge U-Net'], title="Model", loc='upper right')
plt.savefig('{}/COVID19_lgraph.pdf'.format(TARGET_PATH), dpi=100)

In [ ]:
TARGET_PATH

In [ ]:
# Load Input images and gt masks from Unet
images = np.load("{}/COVID19_Unet2_inputs.npy".format(TARGET_PATH))
masks = np.load("{}/COVID19_Unet2_masks.npy".format(TARGET_PATH))
p1 = np.load("{}/COVID19_Unet2_predicted_masks.npy".format(TARGET_PATH))
p2 = np.load("{}/COVID19_HPF_Unet3_predicted_masks.npy".format(TARGET_PATH))

images.shape, masks.shape, p1.shape, p2.shape

In [ ]:
# Decode from one-hot encode
masks = np.array([np.argmax(m.squeeze(), axis=-1) for m in masks])
p1 = np.array([np.argmax(m.squeeze(), axis=-1) for m in p1])
p2 = np.array([np.argmax(m.squeeze(), axis=-1) for m in p2])


images.shape, masks.shape, p1.shape, p2.shape

In [ ]:
# Load unet predictions
#u_preds = np.load("{}/LUNGS2_Unet_predicted_masks.npy".format(TARGET_PATH))
#u_preds.shape
# Load Gauss Unet model
#model = None
#model = load_model("{}/LUNGS2_Gauss_Unet.h5".format(TARGET_PATH), compile=False)
#model.summary()
# Gauss Unet predictions
#preds = model.predict(x=images, verbose=1)
#preds = np.round(preds,0)
#preds.shape

In [ ]:
def compute_jac(y, yhat):
    intersection = yhat.ravel() * y.ravel()
    union = yhat.ravel() + y.ravel() - intersection
    jacard = (np.sum(intersection)/np.sum(union))
    return jacard

In [ ]:
# Check
#compute_jac(masks[0], masks[0])

In [ ]:
from skimage.segmentation import mark_boundaries, find_boundaries

In [ ]:
a = images[0]
gt = masks[0]
x1 = p1[0]
x2 = p2[0]

In [ ]:
u_preds = p1
preds = p2

In [ ]:
#i = 12
for i in range(20):
    import matplotlib.colors
    norm=plt.Normalize(0,3) # 4 classes including BG
    map_name = matplotlib.colors.LinearSegmentedColormap.from_list("", ["black", "red","green","blue"])


    plt.figure(figsize=(20,10))
    plt.subplot(1,5,1)
    if len(images[i].shape) >= 2:
        plt.grid(False)
        plt.axis('off')
        plt.imshow(images[i].squeeze(), cmap='gray') # 1-channel image
    else:
        plt.grid(False)
        plt.axis('off')
        plt.imshow(images[i]) # 3-channel

    #plt.title('Input Image: {}'.format(i), y=-0.15)
    plt.subplot(1,5,2)
    plt.grid(False)
    plt.axis('off')
    plt.imshow(masks[i].reshape(masks[i].shape[0],masks[i].shape[1]), norm=norm, cmap=map_name)
    #plt.title('Ground Truth', y=-0.15)

    plt.subplot(1,5,3)
    plt.grid(False)
    plt.axis('off')
    plt.imshow(u_preds[i].reshape(u_preds[i].shape[0],u_preds[i].shape[1]), norm=norm, cmap=map_name)

    jac = compute_jac(masks[i], u_preds[i])
    #plt.title('U-Net ({0:.4f})'.format(jac), y=-0.15)

    plt.subplot(1,5,4)
    plt.grid(False)
    plt.axis('off')
    plt.imshow(preds[i].reshape(preds[i].shape[0],preds[i].shape[1]), norm=norm, cmap=map_name)

    jac = compute_jac(masks[i], preds[i])
    #plt.title('Edge U-Net ({0:.4f})'.format(jac), y=-0.15)
    plt.tight_layout()
    plt.show()
    plt.close()

In [ ]:
def plot(i, legend_pos, name):
    
    import matplotlib.colors
    norm=plt.Normalize(0,3) # 4 classes including BG
    map_name = matplotlib.colors.LinearSegmentedColormap.from_list("", ["black", "red","green","blue"])
    
    plt.figure(figsize=(22,8))
    plt.subplot(1,4,1)
    if len(images[i].shape) >= 2:
        plt.grid(False)
        plt.axis('off')
        plt.imshow(images[i].squeeze(), cmap='gray') # 1-channel image
    else:
        plt.grid(False)
        plt.axis('off')
        plt.imshow(images[i]) # 3-channel

    #plt.title('a) Input Image', y=-0.1)
    plt.subplot(1,4,2)
    plt.grid(False)
    plt.axis('off')
    plt.imshow(masks[i].reshape(masks[i].shape[0],masks[i].shape[1]), norm=norm, cmap=map_name)
    
    # Plot boundary on image
    legend_elements = [Line2D([3], [3], color='r', lw=3, label='Ground Glass'),
                       Line2D([0], [0], color='g', lw=3, label='Consolidation'),
                       Line2D([0], [0], color='b', lw=3, label='Pleural Effusion')
                      ]
    
    plt.legend(handles=legend_elements, loc=legend_pos, fontsize='large')
    
    #plt.title('b) Ground Truth', y=-0.1)

    plt.subplot(1,4,3)
    plt.grid(False)
    plt.axis('off')
    plt.legend(handles=legend_elements, loc=legend_pos, fontsize='large')
    plt.imshow(u_preds[i].reshape(u_preds[i].shape[0],u_preds[i].shape[1]), norm=norm, cmap=map_name)

    jac = compute_jac(masks[i], u_preds[i])
    #plt.title('c) U-Net', y=-0.1) # ({0:.4f})'.format(jac)

    plt.subplot(1,4,4)
    plt.grid(False)
    plt.axis('off')
    plt.imshow(preds[i].reshape(preds[i].shape[0],preds[i].shape[1]), norm=norm, cmap=map_name)

    jac = compute_jac(masks[i], preds[i])
    #plt.title('d) Edge U-Net', y=-0.1) # ({0:.4f})'.format(jac)    
    plt.legend(handles=legend_elements, loc=legend_pos, fontsize='large')

    #ct = 5
    #edges_pz = mark_boundaries(images[i], masks[i].squeeze().astype(np.int),color=(1,0,0), mode='thin') # gt
    #edges_pz = mark_boundaries(edges_pz, u_preds[i].squeeze().astype(np.int),color=(0,1,0), mode='thin') # unet
    #edges_pz = mark_boundaries(edges_pz, preds[i].squeeze().astype(np.int),color=(0,0,1), mode='thin') # det unet
    #plt.subplot(1,5,5)
    #plt.grid(False)
    #plt.axis('off')

    #legend_pos = legend_pos

    #plt.legend(handles=legend_elements, loc=legend_pos, fontsize='small')
    #plt.title('Comparison', y=-0.15)
    #plt.imshow(edges_pz)

    plt.tight_layout()
    plt.savefig('{}/{}.png'.format(TARGET_PATH, name), bbox_inches='tight', pad_inches=0, dpi=100)
    plt.show()
    plt.close()

In [ ]:
i = 3 # 14 15 19
name = "COVID_multiseg_vis4"
plot(i, 'upper left', name)

In [ ]:
def plot(i, legend_pos, name):
    
    import matplotlib.colors
    norm=plt.Normalize(0,3) # 4 classes including BG
    map_name = matplotlib.colors.LinearSegmentedColormap.from_list("", ["black", "red","green","blue"])
    
    plt.figure(figsize=(22,8))
#     plt.subplot(1,3,1)
#     if len(images[i].shape) >= 2:
#         plt.grid(False)
#         plt.axis('off')
#         plt.imshow(images[i].squeeze(), cmap='gray') # 1-channel image
#     else:
#         plt.grid(False)
#         plt.axis('off')
#         plt.imshow(images[i]) # 3-channel

    #plt.title('a) Input Image', y=-0.1)
    plt.subplot(1,3,1)
    plt.grid(False)
    plt.axis('off')
    plt.imshow(masks[i].reshape(masks[i].shape[0],masks[i].shape[1]), norm=norm, cmap=map_name)
    
    # Plot boundary on image
    legend_elements = [Line2D([3], [3], color='r', lw=3, label='Ground Glass'),
                       Line2D([0], [0], color='g', lw=3, label='Consolidation'),
                       Line2D([0], [0], color='b', lw=3, label='Pleural Effusion')
                      ]
    
    #plt.legend(handles=legend_elements, loc=legend_pos, fontsize=18)
    
    #plt.title('b) Ground Truth', y=-0.1)

    plt.subplot(1,3,2)
    plt.grid(False)
    plt.axis('off')
    #plt.legend(handles=legend_elements, loc=legend_pos, fontsize='large')
    plt.imshow(u_preds[i].reshape(u_preds[i].shape[0],u_preds[i].shape[1]), norm=norm, cmap=map_name)

    jac = compute_jac(masks[i], u_preds[i])
    #plt.title('c) U-Net', y=-0.1) # ({0:.4f})'.format(jac)

    plt.subplot(1,3,3)
    plt.grid(False)
    plt.axis('off')
    plt.imshow(preds[i].reshape(preds[i].shape[0],preds[i].shape[1]), norm=norm, cmap=map_name)

    jac = compute_jac(masks[i], preds[i])
    #plt.title('d) Edge U-Net', y=-0.1) # ({0:.4f})'.format(jac)    
    #plt.legend(handles=legend_elements, loc=legend_pos, fontsize='large')

    #ct = 5
    #edges_pz = mark_boundaries(images[i], masks[i].squeeze().astype(np.int),color=(1,0,0), mode='thin') # gt
    #edges_pz = mark_boundaries(edges_pz, u_preds[i].squeeze().astype(np.int),color=(0,1,0), mode='thin') # unet
    #edges_pz = mark_boundaries(edges_pz, preds[i].squeeze().astype(np.int),color=(0,0,1), mode='thin') # det unet
    #plt.subplot(1,5,5)
    #plt.grid(False)
    #plt.axis('off')

    #legend_pos = legend_pos

    #plt.legend(handles=legend_elements, loc=legend_pos, fontsize='small')
    #plt.title('Comparison', y=-0.15)
    #plt.imshow(edges_pz)

    plt.tight_layout()
    plt.savefig('{}/{}.pdf'.format(TARGET_PATH, name), bbox_inches='tight', pad_inches=0, dpi=300)
    plt.show()
    plt.close()
    
    
i = 19 #14 15 19
name = "COVID_multiseg_vis3"
plot(i, 'upper left', name)

In [ ]:
u_preds = p1
preds = p2

masks.shape, u_preds.shape, preds.shape

In [ ]:
y_true = masks.ravel() 
y_1 = u_preds.ravel() 
y_2 = preds.ravel() 

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score

In [ ]:
def get_res(y, yhat):
    
    fpr, tpr, thresholds = roc_curve((y), yhat)
    AUC_ROC = roc_auc_score(y, yhat)
    
    y_scores= yhat
    y_true = y 
    
    #Confusion matrix
    threshold_confusion = 0.5
    #print ("\nConfusion matrix:  Custom threshold (for positive) of " +str(threshold_confusion))
    y_pred = np.empty((y_scores.shape[0]))
    for i in range(y_scores.shape[0]):
        if y_scores[i]>=threshold_confusion:
            y_pred[i]=1
        else:
            y_pred[i]=0
    confusion = confusion_matrix(y_true, y_pred)
    #print (confusion)
    accuracy = 0
    if float(np.sum(confusion))!=0:
        accuracy = float(confusion[0,0]+confusion[1,1])/float(np.sum(confusion))
    #print ("Global Accuracy: " +str(accuracy))
    specificity = 0
    if float(confusion[0,0]+confusion[0,1])!=0:
        specificity = float(confusion[0,0])/float(confusion[0,0]+confusion[0,1])
    #print ("Specificity: " +str(specificity))
    sensitivity = 0
    if float(confusion[1,1]+confusion[1,0])!=0:
        sensitivity = float(confusion[1,1])/float(confusion[1,1]+confusion[1,0])
    #print ("Sensitivity: " +str(sensitivity))
    precision = 0
    if float(confusion[1,1]+confusion[0,1])!=0:
        precision = float(confusion[1,1])/float(confusion[1,1]+confusion[0,1])
    #print ("Precision: " +str(precision))
    

    #F1 score
    F1_score = f1_score(y_true, y_scores, labels=None, average='binary', sample_weight=None)
    #print ("\nF1 score (F-measure): " +str(F1_score))
    
    return round(F1_score, 4), round(sensitivity, 4), round(specificity, 4), round(accuracy, 4), round(AUC_ROC, 4)
    

In [ ]:
# Unet
F1_score, sensitivity, specificity, accuracy, AUC_ROC = get_res(y_true, y_1)
print(F1_score, sensitivity, specificity, accuracy, AUC_ROC)

In [ ]:
# Deterministic U-Net
F1_score, sensitivity, specificity, accuracy, AUC_ROC = get_res(y_true, y_2)
print(F1_score, sensitivity, specificity, accuracy, AUC_ROC)

In [ ]:
#Area under the ROC curve
fpr1, tpr1, thresholds1 = roc_curve((y_true), y_1)
AUC_ROC1 = roc_auc_score(y_true, y_1)
print ("Area under the ROC curve: " +str(AUC_ROC1))


#Area under the ROC curve
fpr2, tpr2, thresholds2 = roc_curve((y_true), y_2)
AUC_ROC2 = roc_auc_score(y_true, y_2)
print ("Area under the ROC curve: " +str(AUC_ROC2))

In [ ]:
# Best model ROC curve

lw = None
plt.figure()
labels = ['U-Net', 'Edge U-Net']
plt.plot(fpr1, tpr1, color='r', lw=lw, label='U-Net (AUC = {0:.2f}%)'.format(AUC_ROC1 * 100))
plt.plot(fpr2, tpr2, color='b', lw=lw, label='Edge U-Net (AUC = {0:.2f}%)'.format(AUC_ROC2 * 100))

plt.plot([0,1], [0,1], color='orange', linestyle='--')

#plt.xlim([0.0, 1.0])
#plt.ylim([0.0, 1.05])

plt.grid(True)

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel('False Positive Rate', fontsize=15)

plt.title('Endoscopy Dataset: CVC-ClinicDB')
plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel('True Positive Rate', fontsize=15)
plt.legend(prop={'size':10}, loc="lower right")
plt.savefig('{}/CT_roc.pdf'.format(TARGET_PATH), dpi=100)
plt.show()